In [1]:
from dotenv import load_dotenv
from openai import OpenAI
import json
import os
import requests
from pypdf import PdfReader
import gradio as gr

In [2]:
load_dotenv(override=True)
endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
api_key = os.getenv("AZURE_OPENAI_API_KEY")

openai = OpenAI(
    base_url=f"{endpoint}",
    api_key=api_key
)

python-dotenv could not parse statement starting at line 6
python-dotenv could not parse statement starting at line 7


In [3]:
# For pushover

pushover_user = os.getenv("PUSHOVER_USER")
pushover_token = os.getenv("PUSHOVER_TOKEN")
pushover_url = "https://api.pushover.net/1/messages.json"

if pushover_user:
    print(f"Pushover user found and starts with {pushover_user[0]}")
else:
    print("Pushover user not found")

if pushover_token:
    print(f"Pushover token found and starts with {pushover_token[0]}")
else:
    print("Pushover token not found")

Pushover user found and starts with u
Pushover token found and starts with a


In [4]:
def push(messaege: str):
    print("Sending push notification...")
    data = {
        "token": pushover_token,
        "user": pushover_user,
        "message": messaege
    }
    response = requests.post(pushover_url, data=data)
    print(f"Push notification sent with status code {response.status_code}")

In [6]:
def record_user_detail(name: str, email: str, feedback: str):
    # Basic hygiene to avoid empty/whitespace-only inputs
    name = name.strip()
    email = email.strip()
    feedback = feedback.strip()

    if not name or not email or not feedback:
        return {"ok": False, "message": "name, email, and feedback are required"}
    if "@" not in email:
        return {"ok": False, "message": "email appears invalid"}

    push_error = None
    try:
        push(f"New feedback received from {name}({email}): {feedback}")
    except Exception as exc:
        push_error = str(exc)  # Log push failure but continue to persist feedback

    try:
        with open("feedback.txt", "a") as f:
            f.write(f"Name: {name}\nEmail: {email}\nFeedback: {feedback}\n\n")
    except Exception as exc:
        return {
            "ok": False,
            "message": "failed to save feedback",
            "push_error": push_error,
            "error": str(exc),
        }

    return {"ok": True, "message": "Thank you for your feedback!", "push_error": push_error}


In [7]:
def record_unknown_question(question: str):
    question = question.strip()

    if not question:
        return {"ok": False, "message": "question is required"}

    push_error = None
    try:
        push(f"New unknown question received: {question}")
    except Exception as exc:
        push_error = str(exc)  # Log push failure but continue to persist question

    try:
        with open("unknown_questions.txt", "a") as f:
            f.write(f"Question: {question}\n\n")
    except Exception as exc:
        return {
            "ok": False,
            "message": "failed to save question",
            "push_error": push_error,
            "error": str(exc),
        }

    return {"ok": True, "message": "Your question has been recorded.", "push_error": push_error}

In [9]:
""" turn the record_user_detail(name: str, email: str, feedback: str) function into a JSON object 
for the llm to understand the parameters and their types. """
record_user_detail_schema = {
    "name": "record_user_detail",
    "description": "Records user feedback and sends a push notification.",
    "parameters": {
        "type": "object",
        "properties": {
            "name": {
                "type": "string",
                "description": "The name of the user providing feedback.",
            },
            "email": {
                "type": "string",
                "description": "The email address of the user providing feedback.",
            },
            "feedback": {
                "type": "string",
                "description": "The feedback provided by the user.",
            },
        },
        "required": ["name", "email", "feedback"],
    },
}


In [11]:
""" turn the record_unknown_question(question: str) function into a JSON object 
for the llm to understand the parameters and their types. """
record_unknown_question_schema = {
    "name": "record_unknown_question",
    "description": "Records an unknown question that you cannot answer and sends a push notification.",
    "parameters": {
        "type": "object",
        "properties": {
            "question": {
                "type": "string",
                "description": "The unknown question to be recorded.",
            },
        },
        "required": ["question"],
    },
}

In [12]:
tools = [{"type": "function", "function": record_unknown_question_schema},
         {"type": "function", "function": record_user_detail_schema}]

In [13]:
tools

[{'type': 'function',
  'function': {'name': 'record_unknown_question',
   'description': 'Records an unknown question that you cannot answer and sends a push notification.',
   'parameters': {'type': 'object',
    'properties': {'question': {'type': 'string',
      'description': 'The unknown question to be recorded.'}},
    'required': ['question']}}},
 {'type': 'function',
  'function': {'name': 'record_user_detail',
   'description': 'Records user feedback and sends a push notification.',
   'parameters': {'type': 'object',
    'properties': {'name': {'type': 'string',
      'description': 'The name of the user providing feedback.'},
     'email': {'type': 'string',
      'description': 'The email address of the user providing feedback.'},
     'feedback': {'type': 'string',
      'description': 'The feedback provided by the user.'}},
    'required': ['name', 'email', 'feedback']}}}]